## Analyse Benchmark Testbed

In [33]:
%load_ext autoreload
import trafpy
import trafpy.generator as tpg

import os
import pickle

DATA_NAME = 'university_k_4_L_2_n_4_chancap500_numchans1_mldat2e6_bidirectional'
# DATA_NAME = 'ndf50_mldat6e6_load0.4_university'
path_to_benchmark_data = '/scratch/datasets/trafpy/management/flowcentric/{}_testbed_data_v33.obj'.format(DATA_NAME)
filehandler = open(path_to_benchmark_data, 'rb')
tb_dict = pickle.load(filehandler)

print(tb_dict.keys())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dict_keys(['benchmark_data', 'benchmarks', 'envs', 'config'])


In [34]:
%autoreload
# unpack env dicts into list of env objects
envs = tb_dict['envs']

In [36]:
%autoreload
import trafpy
from trafpy.manager import EnvAnalyser, EnvsPlotter

# analyse
analysers = [EnvAnalyser(env, env.scheduler.scheduler_name) for env in envs]
for analyser in analysers:
    analyser.compute_metrics(measurement_start_time='auto',
                             measurement_end_time='auto',
                             print_summary=True)



-=-=-=-=-=-=--= Summary -=-=-=-=-=-=-=-

 ~* General Information *~
Simulation name: 'benchmark_university_load_0.4_repeat_0_scheduler_srpt_v2'
Measurement duration: 2357521.938000072 (Start time : 261946.88200000802 | End time: 2619468.82000008)
Total number of generated demands (jobs or flows) passed to env: 169918
Total number of these demands which arrived during measurement period: 152978
Total info arrived: 3777570484.0 info unit demands arrived
Total info transported: 3746617762.0 info unit demands transported
Load (abs): 1602.3479667826891 info unit demands arrived per unit time (from first to last flow arriving)
Load (frac): 0.4005869916956723 fraction of network capacity requested (from first to last flow arriving)
Throughput (abs): 1589.2186204546301 info units transported per unit time
Throughput (frac): 0.9918061828015914 fraction of arrived info successfully transported
T-Score: 50.028863659977944

 ~* Flow Information *~
Total number of generated flows passed to env (s



-=-=-=-=-=-=--= Summary -=-=-=-=-=-=-=-

 ~* General Information *~
Simulation name: 'benchmark_university_load_0.5_repeat_0_scheduler_srpt_v2'
Measurement duration: 3569986.8467999864 (Start time : 396665.2051999985 | End time: 3966652.051999985)
Total number of generated demands (jobs or flows) passed to env: 308942
Total number of these demands which arrived during measurement period: 277884
Total info arrived: 7126625510.0 info unit demands arrived
Total info transported: 6727225773.0 info unit demands transported
Load (abs): 1996.2609992213452 info unit demands arrived per unit time (from first to last flow arriving)
Load (frac): 0.4990652498053363 fraction of network capacity requested (from first to last flow arriving)
Throughput (abs): 1884.3839099939694 info units transported per unit time
Throughput (frac): 0.9439566823822064 fraction of arrived info successfully transported
T-Score: 52.068731817985416

 ~* Flow Information *~
Total number of generated flows passed to env (



-=-=-=-=-=-=--= Summary -=-=-=-=-=-=-=-

 ~* General Information *~
Simulation name: 'benchmark_university_load_0.4_repeat_0_scheduler_lambda0.95_share'
Measurement duration: 2357521.938000072 (Start time : 261946.88200000802 | End time: 2619468.82000008)
Total number of generated demands (jobs or flows) passed to env: 169918
Total number of these demands which arrived during measurement period: 152978
Total info arrived: 3777570484.0 info unit demands arrived
Total info transported: 3751074646.0 info unit demands transported
Load (abs): 1602.3479667826891 info unit demands arrived per unit time (from first to last flow arriving)
Load (frac): 0.4005869916956723 fraction of network capacity requested (from first to last flow arriving)
Throughput (abs): 1591.1091156937882 info units transported per unit time
Throughput (frac): 0.9929860109527476 fraction of arrived info successfully transported
T-Score: 50.03114881806147

 ~* Flow Information *~
Total number of generated flows passed t



-=-=-=-=-=-=--= Summary -=-=-=-=-=-=-=-

 ~* General Information *~
Simulation name: 'benchmark_university_load_0.3_repeat_0_scheduler_lambda0.7_share'
Measurement duration: 2814172.8914995966 (Start time : 312685.8768332885 | End time: 3126858.768332885)
Total number of generated demands (jobs or flows) passed to env: 154471
Total number of these demands which arrived during measurement period: 138977
Total info arrived: 3406784261.0 info unit demands arrived
Total info transported: 3274882161.0 info unit demands transported
Load (abs): 1210.581009891193 info unit demands arrived per unit time (from first to last flow arriving)
Load (frac): 0.3026452524727982 fraction of network capacity requested (from first to last flow arriving)
Throughput (abs): 1163.7103643816652 info units transported per unit time
Throughput (frac): 0.9612825204372399 fraction of arrived info successfully transported
T-Score: 49.90655762575113

 ~* Flow Information *~
Total number of generated flows passed to



-=-=-=-=-=-=--= Summary -=-=-=-=-=-=-=-

 ~* General Information *~
Simulation name: 'benchmark_university_load_0.2_repeat_0_scheduler_lambda0.9_share'
Measurement duration: 4907976.906375021 (Start time : 545330.7673750024 | End time: 5453307.673750023)
Total number of generated demands (jobs or flows) passed to env: 169918
Total number of these demands which arrived during measurement period: 152860
Total info arrived: 3946210245.0 info unit demands arrived
Total info transported: 4005921177.0 info unit demands transported
Load (abs): 804.0401004891094 info unit demands arrived per unit time (from first to last flow arriving)
Load (frac): 0.20101002512227734 fraction of network capacity requested (from first to last flow arriving)
Throughput (abs): 816.2061992990775 info units transported per unit time
Throughput (frac): 1.015131209006326 fraction of arrived info successfully transported
T-Score: 52.6933595971542

 ~* Flow Information *~
Total number of generated flows passed to en



-=-=-=-=-=-=--= Summary -=-=-=-=-=-=-=-

 ~* General Information *~
Simulation name: 'benchmark_university_load_0.2_repeat_0_scheduler_lambda0.05_share'
Measurement duration: 4907976.906375021 (Start time : 545330.7673750024 | End time: 5453307.673750023)
Total number of generated demands (jobs or flows) passed to env: 169918
Total number of these demands which arrived during measurement period: 152860
Total info arrived: 3946210245.0 info unit demands arrived
Total info transported: 4006976473.0 info unit demands transported
Load (abs): 804.0401004891094 info unit demands arrived per unit time (from first to last flow arriving)
Load (frac): 0.20101002512227734 fraction of network capacity requested (from first to last flow arriving)
Throughput (abs): 816.4212157957178 info units transported per unit time
Throughput (frac): 1.0153986291219514 fraction of arrived info successfully transported
T-Score: 52.6909079507936

 ~* Flow Information *~
Total number of generated flows passed to 



-=-=-=-=-=-=--= Summary -=-=-=-=-=-=-=-

 ~* General Information *~
Simulation name: 'benchmark_university_load_0.5_repeat_0_scheduler_lambda0.1_share'
Measurement duration: 3569986.8467999864 (Start time : 396665.2051999985 | End time: 3966652.051999985)
Total number of generated demands (jobs or flows) passed to env: 308942
Total number of these demands which arrived during measurement period: 277884
Total info arrived: 7126625510.0 info unit demands arrived
Total info transported: 6833427250.0 info unit demands transported
Load (abs): 1996.2609992213452 info unit demands arrived per unit time (from first to last flow arriving)
Load (frac): 0.4990652498053363 fraction of network capacity requested (from first to last flow arriving)
Throughput (abs): 1914.1323324833113 info units transported per unit time
Throughput (frac): 0.9588587530537998 fraction of arrived info successfully transported
T-Score: 51.65657636639852

 ~* Flow Information *~
Total number of generated flows passed t



-=-=-=-=-=-=--= Summary -=-=-=-=-=-=-=-

 ~* General Information *~
Simulation name: 'benchmark_university_load_0.1_repeat_0_scheduler_lambda0.1_share'
Measurement duration: 8668532.64149981 (Start time : 963170.2934999788 | End time: 9631702.934999788)
Total number of generated demands (jobs or flows) passed to env: 154471
Total number of these demands which arrived during measurement period: 139161
Total info arrived: 3498030177.0 info unit demands arrived
Total info transported: 3496183796.0 info unit demands transported
Load (abs): 403.53198420843444 info unit demands arrived per unit time (from first to last flow arriving)
Load (frac): 0.10088299605210861 fraction of network capacity requested (from first to last flow arriving)
Throughput (abs): 403.3189861064073 info units transported per unit time
Throughput (frac): 0.9994721655027049 fraction of arrived info successfully transported
T-Score: 51.49911053883083

 ~* Flow Information *~
Total number of generated flows passed to 

In [37]:
# TEMPORARY CELL FOR RENAMING ANY PREVIOUSLY SAVED SCHEDULER NAMES WITH NEW SHORTENED CONVENTION
for analyser in analysers:
    if analyser.env.scheduler.scheduler_name == 'fair_share':
        analyser.env.scheduler.scheduler_name = 'FS'
    elif analyser.env.scheduler.scheduler_name == 'first_fit':
        analyser.env.scheduler.scheduler_name = 'FF'
    elif analyser.env.scheduler.scheduler_name == 'random':
        analyser.env.scheduler.scheduler_name = 'Rand'
    elif analyser.env.scheduler.scheduler_name == 'srpt_v2':
        analyser.env.scheduler.scheduler_name = 'SRPT'
    elif analyser.env.scheduler.scheduler_name[0:6] == 'lambda':
        analyser.env.scheduler.scheduler_name = '\u03BB' + str(analyser.env.scheduler._lambda) + 'S'
    analyser.subject_class_name = analyser.env.scheduler.scheduler_name

In [41]:
%autoreload
import trafpy
from trafpy.manager import EnvAnalyser, EnvsPlotter
    
# plot
plotter = EnvsPlotter()
_ = plotter.display_t_score_table(*analysers, display_table=True)
# _ = plotter.plot_t_score_scatter(*analysers, gridlines=True, figsize=(4, 3), legend_ncol=2)

# COMMENT IF HAVEN'T TRACKED QUEUE EVOLUTION AND SCHEDULER DECISION COLOUR GRID
# _ = plotter.plot_src_dst_queue_evolution_for_different_loads('server_2', 'server_10', 'queue_lengths_num_flows', *analysers)
# _ = plotter.plot_src_dst_queue_evolution_for_different_loads('server_2', 'server_10', 'queue_lengths_info_units', *analysers)
# _ = plotter.plot_demand_slot_colour_grid_for_different_schedulers(*analysers)

# COMMENT IF WANT TO SAVE TIME
# _ = plotter.plot_link_utilisation_vs_time_for_different_loads(*analysers, mean_period=2500, figsize=(6, 2), legend_ncol=2, plot_legend=False)
# _ = plotter.plot_link_concurrent_demands_vs_time_for_different_loads(*analysers, mean_period=500)




# _ = plotter.plot_demand_completion_time_vs_size_for_different_loads(*analysers, gridlines=True, figsize=(4, 3), legend_ncol=2)
# _ = plotter.plot_throughput_vs_load(*analysers, plot_bar_charts=False, gridlines=True, figsize=(6, 2), legend_ncol=2)
# _ = plotter.plot_fcts_cdf_for_different_loads(*analysers, gridlines=True, figsize=(6, 2), legend_ncol=2)
# _ = plotter.plot_mean_fct_vs_load(*analysers, gridlines=True, figsize=(6, 2), legend_ncol=2)
# _ = plotter.plot_99th_percentile_fct_vs_load(*analysers, gridlines=True, figsize=(6, 2), legend_ncol=2)
# _ = plotter.plot_max_fct_vs_load(*analysers, gridlines=True, figsize=(6, 2), legend_ncol=2)
# _ = plotter.plot_fraction_of_arrived_flows_dropped_vs_load(*analysers, gridlines=True, cdf_figsize=(6, 2), scatter_figsize=(4,3), legend_ncol=2)
# _ = plotter.plot_fraction_of_arrived_info_dropped_vs_load(*analysers, gridlines=True, cdf_figsize=(6, 2), scatter_figsize=(4,3), legend_ncol=2)

,Load,Subject,T-Score,Mean FCT,p99 FCT,Max FCT,Throughput Rate,Flows Dropped,Info Dropped
0,0.1,SRPT,51.587609,127.6,1516.3,329350.5,403.4,0.000014,0.00142
1,0.1,λ0.95S,51.587544,127.9,1521.8,329250.5,403.4,0.000014,0.00142
2,0.1,λ0.9S,51.587365,128.3,1527.4,329300.5,403.4,0.000014,0.00142
3,0.1,λ0.7S,51.586846,129.6,1548.7,328950.5,403.4,0.000014,0.00142
4,0.1,λ0.5S,51.586221,131.4,1572.5,328000.5,403.4,0.000014,0.00142
5,0.1,λ0.3S,51.499653,133.7,1620.5,326350.5,403.3,0.000022,0.00172
6,0.1,λ0.1S,51.499111,136.9,1650.7,323500.5,403.3,0.000022,0.00172
7,0.1,λ0.05S,51.498782,137.9,1677.0,323200.5,403.3,0.000022,0.00172
8,0.1,FS,51.498464,139.0,1695.1,322800.5,403.3,0.000022,0.00172
9,0.2,λ0.9S,52.693360,155.0,1752.4,601680.4,816.2,0.000033,0.00206


In [6]:
# %autoreload
# import trafpy
# import trafpy.generator as tpg
# from trafpy.manager import RandomAgent, RWA, Demand, DCN, EnvAnalyser
# import json

# DATA_BAME = 'artificial_light_chancap10'
# path_to_benchmark_data = os.path.dirname(trafpy.__file__)+'/../data/benchmark_data/{}_benchmark_data.json'.format(DATA_NAME)
# benchmark_data = json.loads(tpg.load_data_from_json(path_to_benchmark_data))
# benchmarks = list(benchmark_data.keys())

# SLOT_SIZE = 1.0
# PACKET_SIZE = 1
# NUM_CHANNELS = 1
# NUM_K_PATHS = 1
# MAX_FLOWS = 10
# MAX_TIME = None

In [7]:
# network = tpg.gen_fat_tree(k=3, N=2, num_channels=1, server_to_rack_channel_capacity=10)
# rwa = RWA(tpg.gen_channel_names(NUM_CHANNELS), NUM_K_PATHS)
# scheduler = RandomAgent(network, rwa, slot_size=SLOT_SIZE, packet_size=PACKET_SIZE)

# num_benchmark_tests = 0
# for benchmark in benchmarks:
#     for load in benchmark_data[benchmark]:
#         for repeat in benchmark_data[benchmark][load]:
#             num_benchmark_tests += 1

# for benchmark in benchmarks:
#     for load in list(benchmark_data[benchmark].keys()):
#         for repeat in benchmark_data[benchmark][load]:
#             if json.loads(load) == 0.1 and scheduler.scheduler_name == 'random':
#                 demand_data = benchmark_data[benchmark][load][repeat]
#                 demand = tpg.Demand(demand_data)
#                 env = DCN(network, demand, scheduler, num_k_paths=NUM_K_PATHS, slot_size=SLOT_SIZE, max_flows=MAX_FLOWS, max_time=MAX_TIME)
#                 print(env.slot_size)
                
#                 observation = env.reset()
#                 scheduler.register_env(env)
                
#                 while True:
#                     action = scheduler.get_action(observation)
#                     observation, reward, done, info = env.step(action)
                    
#                     if done:
#                         print('Completed')
#                         analyser = EnvAnalyser(env)
#                         analyser.compute_metrics(print_summary=True)

In [8]:
# packet size = info
# slot size = sec
# min bandwidth in network = info per sec

# assume that 1 packet size of info can be transferred per sub slot
# info per sub slot = 1 packet = packet size
# sub slot size = (info per sub slot / min bandwidth in network) * slot size

In [9]:
a = None
if a is not None:
    print('not none')
else:
    print('None')

None


In [10]:
increment_time = 1.001
print(round(1-(increment_time-1),6))

0.999


In [11]:
ep_link_capacity = 500
